In [ ]:
import sys

In [ ]:
from jupyter.experiment import Experiment
from jupyter.profile import Profile

In [ ]:
EXPERIMENT_ID_SWAN = sys.stdin.readline().strip()

In [ ]:
IP = '127.0.0.1'
PORT = 9042

In [ ]:
exp1 = Experiment(cassandra_cluster=[IP], experiment_id=EXPERIMENT_ID_SWAN, port=PORT, name="experiment 1",
                  dir_csv='../test_data')

In [ ]:
assert exp1.cached_experiment == '../test_data/%s' % EXPERIMENT_ID_SWAN
assert exp1.name == 'experiment 1'
assert exp1.CASSANDRA_SESSION != None

In [ ]:
p1 = Profile(exp1, slo=500)
p1.sensitivity_table(show_throughput=True)

In [ ]:
assert p1.slo == 500
assert p1.categories == ['Baseline', 'Caffe']
assert p1.latency_qps_aggrs.keys() == ['x', 'slo', 'Caffe', 'Baseline']